In [ ]:
import pandas as pd
import re
emotion_data = pd.read_csv("ratings_train.txt", encoding='utf-8', sep='\t')
emotion_data_test = pd.read_csv("ratings_test.txt", encoding='utf-8', sep='\t')
emotion_data = emotion_data.dropna()
emotion_data_test = emotion_data_test.dropna()

In [ ]:
emotion_data_test.isnull().sum()

,0
id,0
document,0
label,0


In [ ]:
!pip install konlpy
!pip install tensorflow
!pip install keras

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import torch
torch.cuda.is_available()

False

In [ ]:
from konlpy.tag import Okt
okt = Okt()
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다']
data_train = []
data_test = []
for sentence, label in zip(emotion_data['document'], emotion_data['label']):
    data = []
    sentence = [word for word in okt.morphs(re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', sentence), stem=True) if not word in stopwords]
    data.append(sentence)
    data.append(label)
    data_train.append(data)

for sentence, label in zip(emotion_data_test['document'], emotion_data_test['label']):
    data = []
    sentence = [word for word in okt.morphs(re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', sentence), stem=True) if not word in stopwords]
    data.append(sentence)
    data.append(label)
    data_test.append(data)

data_test[:10]

[[['굳다', 'ㅋ'], 1],
 [[], 0],
 [['뭐', '야', '평점', '나쁘다', '않다', '점', '짜다', '리', '더', '더욱', '아니다'], 0],
 [['지루하다', '않다', '완전', '막장', '임', '돈', '주다', '보기', '에는'], 0],
 [['만', '아니다', '별', '다섯', '개', '주다', '왜', '로', '나오다', '제', '심기', '불편하다'], 0],
 [['음악', '주가', '되다', '최고', '음악', '영화'], 1],
 [['진정하다', '쓰레기'], 0],
 [['마치',
   '미국',
   '애니',
   '에서',
   '튀어나오다',
   '창의력',
   '없다',
   '로봇',
   '디자인',
   '부터가',
   '고개',
   '젖다'],
  0],
 [['갈수록',
   '개판',
   '되다',
   '중국영화',
   '유치하다',
   '내용',
   '없다',
   '폼',
   '잡다',
   '끝나다',
   '말',
   '안되다',
   '무기',
   '유치하다',
   '남무',
   '아',
   '그리다',
   '동사서독',
   '같다',
   '영화',
   '이건',
   '류',
   '아',
   '류작',
   '이다'],
  0],
 [['이별', '아픔', '뒤', '찾아오다', '새롭다', '인연', '기쁨', '모든', '사람', '그렇다', '않다'], 1]]

In [ ]:
# data_train, data_test 구조([[토큰리스트], 라벨])에서 토큰 리스트와 라벨 분리
# 주의: data_train[i][0][0] 이 실제 토큰 리스트임
import numpy as np
X_train_tokens = [item[0] for item in data_train]
y_train = np.array([item[1] for item in data_train])
X_test_tokens = [item[0] for item in data_test]
y_test = np.array([item[1] for item in data_test])

# Tokenizer 생성 및 훈련 데이터 기준 단어 집합 생성
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train_tokens)

In [ ]:
# 단어 집합 크기 확인 (OOV 고려 +1)
vocab_size = len(tokenizer.word_index) + 1
print(f"\n어휘 사전 크기: {vocab_size}")


어휘 사전 크기: 43753


In [ ]:
# 텍스트를 정수 시퀀스로 변환
X_train_sequences = tokenizer.texts_to_sequences(X_train_tokens)
X_test_sequences = tokenizer.texts_to_sequences(X_test_tokens)

In [ ]:
# 패딩을 위한 최대 길이 결정 (훈련 데이터 기준)
print("\n리뷰 최대 길이 계산 중...")
max_len = max(len(l) for l in X_train_sequences)
print(f'리뷰의 최대 길이 : {max_len}')


리뷰 최대 길이 계산 중...
리뷰의 최대 길이 : 72


In [ ]:
# 패딩 적용
print("패딩 적용 중...")
X_train_pad = pad_sequences(X_train_sequences, maxlen=max_len, padding='post')
X_test_pad = pad_sequences(X_test_sequences, maxlen=max_len, padding='post')
print("패딩 완료.")

print(f"패딩 후 훈련 데이터 형태 (X): {X_train_pad.shape}")
print(f"훈련 라벨 데이터 형태 (y): {y_train.shape}")
print(f"패딩 후 테스트 데이터 형태 (X): {X_test_pad.shape}")
print(f"테스트 라벨 데이터 형태 (y): {y_test.shape}")

패딩 적용 중...
패딩 완료.
패딩 후 훈련 데이터 형태 (X): (149995, 72)
훈련 라벨 데이터 형태 (y): (149995,)
패딩 후 테스트 데이터 형태 (X): (49997, 72)
테스트 라벨 데이터 형태 (y): (49997,)


In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt # 시각화 추가 (선택 사항)
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split # 검증 데이터 분리용

In [ ]:
X_train_final, X_val, y_train_final, y_val = train_test_split(
    X_train_pad, y_train, test_size=0.2, random_state=42, stratify=y_train
)

print(f"\n최종 훈련 데이터: {X_train_final.shape}, {y_train_final.shape}")
print(f"검증 데이터: {X_val.shape}, {y_val.shape}")
print(f"테스트 데이터: {X_test_pad.shape}, {y_test.shape}")


최종 훈련 데이터: (119996, 72), (119996,)
검증 데이터: (29999, 72), (29999,)
테스트 데이터: (49997, 72), (49997,)


In [ ]:
import tensorflow as tf
# --- 5. LSTM 모델 구축, 컴파일, 학습, 평가, 예측 (이 부분은 이전과 거의 동일) ---

# 모델 파라미터
embedding_dim = 100
lstm_units = 128

# 모델 설계
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len))
model.add(Dropout(0.4))
model.add(Bidirectional(LSTM(units=lstm_units)))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# 콜백 설정
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_nsmc_lstm_model_v2.keras', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True) # 저장 파일명 변경

# 모델 학습
epochs = 20
batch_size = 32

print("\n--- 모델 학습 시작 ---")
history = model.fit(X_train_final, y_train_final,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(X_val, y_val),
                    callbacks=[es, mc],
                    verbose=1)

# 모델 평가
try:
    loaded_model = load_model('best_nsmc_lstm_model_v2.keras') # 변경된 파일명으로 로드
    print("\n--- 베스트 모델 로드 후 테스트 데이터 평가 ---")
    loss, accuracy = loaded_model.evaluate(X_test_pad, y_test, verbose=0)
except Exception as e:
    print(f"\n모델 로드 실패: {e}. 마지막 학습 모델로 평가합니다.")
    loaded_model = model
    print("\n--- 마지막 학습 모델 테스트 데이터 평가 ---")
    loss, accuracy = loaded_model.evaluate(X_test_pad, y_test, verbose=0)

print(f"테스트 손실: {loss:.4f}")
print(f"테스트 정확도: {accuracy:.4f}")

model.summary()


# 예측 함수 정의 (내부 전처리 로직은 사용자 원본과 동일하게 유지)
def predict_sentiment(text, tokenizer, model, max_len):
    # 1. 텍스트 정제 및 토큰화 (사용자 원본 방식 유지)
    processed_text = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', str(text))
    tokenized_text = [word for word in okt.morphs(processed_text, stem=True) if not word in stopwords and len(word) > 1]

    if not tokenized_text:
        return "분석할 유효한 토큰이 없습니다."

    # 2. 정수 인코딩
    try:
        sequence = tokenizer.texts_to_sequences([tokenized_text])
    except Exception as e:
        return f"정수 인코딩 오류: {e}"

    # 3. 패딩
    padded_sequence = pad_sequences(sequence, maxlen=max_len, padding='post')

    # 4. 예측
    try:
        if padded_sequence.shape != (1, max_len):
             padded_sequence = padded_sequence.reshape(1, max_len)
        prediction = model.predict(padded_sequence, verbose=0)
    except Exception as e:
        return f"모델 예측 오류: {e}, 입력 형태: {padded_sequence.shape}"

    # 5. 결과 해석
    score = float(prediction[0][0])
    if score > 0.5:
        return f"긍정"
    else:
        return f"부정"


# 새로운 텍스트 예측 테스트
print("\n--- 새로운 텍스트 예측 ---")
test_sentence_1 = "이 영화 진짜 재밌음 ㅋㅋ 강추!!"
result_1 = predict_sentiment(test_sentence_1, tokenizer, loaded_model, max_len)
print(f"'{test_sentence_1}' -> {result_1}")

test_sentence_2 = "별로였어요. 돈 아까움."
result_2 = predict_sentiment(test_sentence_2, tokenizer, loaded_model, max_len)
print(f"'{test_sentence_2}' -> {result_2}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



--- 모델 학습 시작 ---
Epoch 1/20
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 0s 265ms/step - accuracy: 0.7768 - loss: 0.4508
Epoch 1: val_accuracy improved from -inf to 0.84149, saving model to best_nsmc_lstm_model_v2.keras
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 1084s 287ms/step - accuracy: 0.7768 - loss: 0.4507 - val_accuracy: 0.8415 - val_loss: 0.3549
Epoch 2/20
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step - accuracy: 0.8739 - loss: 0.2987
Epoch 2: val_accuracy improved from 0.84149 to 0.84809, saving model to best_nsmc_lstm_model_v2.keras
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 1106s 288ms/step - accuracy: 0.8739 - loss: 0.2987 - val_accuracy: 0.8481 - val_loss: 0.3473
Epoch 3/20
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step - accuracy: 0.8960 - loss: 0.2525
Epoch 3: val_accuracy improved from 0.84809 to 0.84869, saving model to best_nsmc_lstm_model_v2.keras
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 1057s 282ms/step - accuracy: 0.8960 - loss: 0.2525 - val_accuracy: 0.8487 - val_loss: 0.3584
Epoch 4/20
3750/3750 ━━━━━━━━━━━━━━━━

In [ ]:
# 흥부를 묘사하는 문장 분석
print("\n--- 흥부 텍스트 예측 ---")
Heungbu = ["흥부는 고운 마음씨로 사람들을 도왔다.",
           "흥부는 마음씨 착하고 효행이 지극하며 동기간의 우애가 극진한데",
           "흥부는 충실, 온 후, 인자하였으니, 형의 하는 짓을 탄식하고 때로는 간할 마음을 가져보았으나 말해 보아야 쓸데없으므로 말없이 주면 먹고 시키는 일이나 공손히 하였다.",
           "흥부의 어진 마음은 조금도 변함이 없었다.",
           "흥부는 울며 사정하였다.",
           "흥부는 과연 어진 사람이다. 공 있는 자에게 보은함은 군자의 도리이니, 그 은혜를 어찌 아니 갚으랴？",
           "흥부의 구조를 받아 살아서 돌아왔습니다.",
           "흥부의 가난을 면케 해주신다면 그로써 소신은 그 은공의 만분의 일이라도 갚을까 합니다.",
           "흥부가 거절하자 이번은 화초장이나 달라고 한다.",
           "흥부는 안방을 치우고 형님 내외를 거처케 한 다음 의식을 후히 내어 대접하며 위로하고, 한편으로 좋은 터를 잡아 수만금을 아낌없이 들여 집을 짓되 제 집과 같게 하고 세간이며 의복 음식을 똑같게 하여 그 형을 살게 하여 주었다.",
           "밥 좀 많이 붙은 주걱으로요. 그 밥 갖다가 아이들 구경이나 시키겠소.",
           "흥부는 얼른 받아 제비 새끼의 상한 다리를 곱게 감아 매어 찬 이슬에 얹어 두었다."

]
Heungbu_label = []

for test_sentence_1 in Heungbu:
  result_1 = predict_sentiment(test_sentence_1, tokenizer, loaded_model, max_len)
  if result_1 == "긍정":
    Heungbu_label.append(1)
  else:
    Heungbu_label.append(0)

  print(f"'{test_sentence_1}' -> {result_1}")


# 놀부를 묘사하는 문장 분석
print("\n--- 놀부 텍스트 예측 ---")
Nolbu = ["놀부는 부모에게는 불효이고 동기간에 우애가 조금도 없으니, 그 마음 쓰는 것이 괴상하였다.",
         "놀부는 애초부터 오장에 칠 부였다. 말하자면 심술보가 하나 더 있어 심술보가 한 번만 뒤집히면 심사를 야단스럽게도 피웠다.",
         "이놈의 심사가 이렇듯 모과나무같이 뒤틀리고 동풍 안개 속에 수숫잎, 같이 꼬여 그 흉악함을 헤아릴 수 없었다.",
         "놀부의 악한 마음은 부모가 물려준 많은 재산을 독차지하고 아우 흥부를 구박하나",
        "놀부는 화를 더럭 내어 벼락같은 소리로 하인 마당쇠를 부르는 것이었다.",
        "이놈이 도둑질했나？ 내가 가서 욱대기면 반 재산을 뺏어낼 것이다.",
        "제비 새끼 잡아 두 발목을 지끈등 분지르고는",
        "원수 같은 놀부놈아, 다음 해 춘삼월에 다시 와서 원수를 갚을 것이니 잘 있거라. 지지위 지지.",
        "놀부놈은 기겁을 하여 돈 오천 냥을내주고 겨우 그들을 보내고 나니 열이 치받쳤다.",
         "이놈 놀부야, 돈이 중하냐 목숨이 중하냐？",
         "이놈 놀부야, 네가 세상에 태어나 부모께 불효요, 형제에게 불목하고 친척과 불화하니 죄악이 네 털을 빼어 세어도 당치 못할 것이다.",
         "놀부가 기가 막혀 발을 동동 구르며 탄식하였다.",
         ]

Nolbu_label = []

for test_sentence_2 in Nolbu:
  test_sentence_2 = test_sentence_2
  result_2 = predict_sentiment(test_sentence_2, tokenizer, loaded_model, max_len)
  print(f"'{test_sentence_2}' -> {result_2}")

  if result_2 == "긍정":
    Nolbu_label.append(1)
  else:
    Nolbu_label.append(0)


In [ ]:
heungbu_pos_count = sum(label == 1 for label in Heungbu_label)
heungbu_neg_count = sum(label == 0 for label in Heungbu_label)
nolbu_pos_count = sum(label == 1 for label in Heungbu_label)
nolbu_neg_count = sum(label == 0 for label in Heungbu_label)

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf